# DrakeVisualizer.jl
This notebook demonstrates interacting with the Drake Visualizer app from Julia. To run it, you'll need to:

* install Drake: http://drake.mit.edu/
* add the Drake lcmtypes to your python path: 
    * `export PYTHONPATH="$PYTHONPATH:/path/to/drake-distro/build/lib/python2.7/dist-packages"`
* install the DrakeVisualizer.jl package:
    * `Pkg.clone("https://github.com/rdeits/DrakeVisualizer.jl.git")`
* launch the `drake-visualizer` app:
    * `cd /path/to/drake-distro/build/bin`
    * `./drake-visualizer`
        

In [1]:
# Activate the DrakeVisualizer package, and import some other 
# useful functions
using DrakeVisualizer
import GeometryTypes: HyperRectangle, Vec, HomogenousMesh, SignedDistanceField
import Meshing
import AffineTransforms: tformtranslate, tformrotate
import ColorTypes: RGBA
import Interact: @manipulate

LoadError: LoadError: module DrakeVisualizer not found in current path; you should rm("/Users/rdeits/.julia/declarative/af436fcf2f98e2f612470aec5b00fca6/lib/v0.4/DrakeVisualizer.ji") to remove the orphaned cache file
while loading In[1], in expression starting on line 3

In [2]:
# First, we'll create a simple geometric object
box = HyperRectangle(Vec(0.,0,0), Vec(1.,1,1))

GeometryTypes.HyperRectangle{3,Float64}(FixedSizeArrays.Vec{3,Float64}((0.0,0.0,0.0)),FixedSizeArrays.Vec{3,Float64}((1.0,1.0,1.0)))

In [3]:
# Visualizer() causes the viewer to spawn a geometry or a set of geometries. 
# It returns a Visualizer, which includes all the information
# about that loaded geometry. 
# Note that the model is initially loaded in the zero configuration 
# (that is, its position and rotation are all zeros)
model = Visualizer(box);

In [4]:
# We can use draw() to tell the viewer to draw the box at a specific
# position. tformtranslate() creates an AffineTransform corresponding
# to the given x; y; z translation.
draw(model, [tformtranslate([1;0;0])])

In [5]:
# We can also rotate the model by sending a different AffineTransform
draw(model, [tformrotate([0;0;1], pi/4)])

In [6]:
# Let's look more at what's being done under the hood. The model
# owns a reference to a Robot object. A Robot contains a list of 
# Link objects. Each Link contains a list of GeometryData objects.
# And each GeometryData contains a single geometric primitive as
# well as information about its color and its position with
# respect to the link that contains it.
#
# Calling Visualizer() on a single geometry (the box above) results in 
# a new Robot being automatically created. That Robot has one link,
# and that Link has one GeometryData whose geometry is the box we
# provided. 
model.robot

DrakeVisualizer.Robot([DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{Float64,GeometryTypes.HyperRectangle{3,Float64}}(GeometryTypes.HyperRectangle{3,Float64}(FixedSizeArrays.Vec{3,Float64}((0.0,0.0,0.0)),FixedSizeArrays.Vec{3,Float64}((1.0,1.0,1.0))),AffineTransforms.AffineTransform{Float64,3}:
matrix: 3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
translation: [0.0,0.0,0.0]
,RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")])

In [7]:
model.robot.links

1-element Array{DrakeVisualizer.Link,1}:
 DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{Float64,GeometryTypes.HyperRectangle{3,Float64}}(GeometryTypes.HyperRectangle{3,Float64}(FixedSizeArrays.Vec{3,Float64}((0.0,0.0,0.0)),FixedSizeArrays.Vec{3,Float64}((1.0,1.0,1.0))),AffineTransforms.AffineTransform{Float64,3}:
matrix: 3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
translation: [0.0,0.0,0.0]
,RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")

In [8]:
model.robot.links[1].geometry_data[1].geometry

GeometryTypes.HyperRectangle{3,Float64}(FixedSizeArrays.Vec{3,Float64}((0.0,0.0,0.0)),FixedSizeArrays.Vec{3,Float64}((1.0,1.0,1.0)))

In [9]:
@assert model.robot.links[1].geometry_data[1].geometry === box

In [10]:
# Now let's make some more complicated robots. We'll create a 
# new GeometryData from the box, but color it green this time.
green_box_data = GeometryData(box)
green_box_data.color = RGBA(0., 1, 0, 0.5)
model = Visualizer(green_box_data);

In [11]:
# Now let's create a blue box
blue_box_data = GeometryData(box, tformtranslate([0;0;1.]), RGBA(0,0,1,0.5))

# We can create a new Link that contains two geometries: the 
# green box and the blue box. 
link1 = Link([green_box_data; blue_box_data], "link1")
model = Visualizer(link1);

In [12]:
# And we can draw that new link wherever we like.
draw(model, [tformtranslate([0.75; 0; 0])])

# Note that the green and blue boxes move together. That's
# because they are part of the same rigid link. 

In [13]:
# If we want different parts of our robot to move separately,
# they need to be on separate links. Let's create a second
# Link and then a robot containing both of those links. 
red_box_data = GeometryData(box)
link2 = Link([red_box_data], "link2")
robot = Robot([link1; link2])

# When we load this new robot, the two links are both drawn 
# at position [0; 0; 0]; right on top of each other.
model = Visualizer(robot);


In [14]:
# Now that we have two links, we need to specify two transforms
# for the draw() command. The order of the transforms matches
# the order of the links in "Robot([link1; link2])"
draw(model, [tformtranslate([0.5; 0; 0]), tformtranslate([-0.5;0;0])])

In [15]:
# We can now move the two links independently
@manipulate for x1 in linspace(0, 2), x2 in linspace(0, 2)
    draw(model, [tformtranslate([x1; 0; 0]); tformtranslate([x2; 0; 0])])
end

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x1",0.9795918367346939,linspace(0.0,2.0,50),true)

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x2",0.9795918367346939,linspace(0.0,2.0,50),true)

nothing

In [16]:
# Of course, we can draw much more interesting geometries than 
# just simple boxes. Let's create a triangulated mesh by finding
# the 0-level set of some function. 
# 
# First, we'll define our function:
f = x -> norm(x)^2 - abs(sin(atan2(x[2], x[1])))

# Then we sample that function within the volume from 
# [-1; -1; -1] to [1; 1; 1]
sdf = SignedDistanceField(f, HyperRectangle(Vec(-1.,-1,-1), Vec(2.,2,2)))

# Next we find a mesh describing the 0-level set of that function
# using the marching tetrahedra algorithm
mesh = HomogenousMesh(sdf)

# And we can now load that mesh into the visualizer
model = Visualizer(mesh);

In [17]:
# We can even manipulate the geometry by changing the function:
@manipulate for iso_level in linspace(-1, 1)
    f = x -> norm(x)^2 - abs(sin(atan2(x[2], x[1]))) - iso_level
    sdf = SignedDistanceField(f, HyperRectangle(Vec(-1.1,-1.1,-1.1), Vec(2.2,2.2,2.2)))
    mesh = HomogenousMesh(sdf)
    model = Visualizer(mesh);
end

Interact.Slider{Float64}(Signal{Float64}(-0.02040816326530612, nactions=0),"iso_level",-0.02040816326530612,linspace(-1.0,1.0,50),true)

DrakeVisualizer.VisualizerModel(DrakeVisualizer.Robot([DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{Float64,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 4900xFixedSizeArrays.Point{3,Float64},     faces: 9792xGeometryTypes.Face{3,Int64,0}, )
,AffineTransforms.AffineTransform{Float64,3}:
matrix: 3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
translation: [0.0,0.0,0.0]
,RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")]),DrakeVisualizer.Visualizer(PyLCM.LCM(PyObject <LCM object at 0x31e60ede0>)),1)